# Install requirements

In [ ]:
pip install matplotlib numpy opencv-python argparse

# Import Libs

In [ ]:
import os
import cv2 as cv2
import numpy as np
import csv
import numpy
import sys
import logging
import argparse
import json

## Just run the Code.

Folder Structure:

- Data
   \- stacks         Insert the "haystack" Images here
    - needle         The "Needle" we want to find in the Haystack
    - output         Storage Folder for the processed Images
    - output_csv     Log Folder for txt,json and csv files
        
Haystack Images should be larger then the needle Images. I used around 4k resolution.
Put as many images as you like in the stacks Folder.
Importand is, that the needle Image has to be in the same size because the script don't resize the needle.

Look at the demo Image for an example.

If something goes wrong, set __DEBUG = True


In [ ]:
 # Copyright (C) 2021 Stefan Knaak - All Rights Reserved
 # You may use, distribute and modify this code under the
 # terms of MIT License.
 #
 #	MIT License
 #
 #	Copyright (c) 2021 Stefan Knaak
 #
 #	Permission is hereby granted, free of charge, to any person obtaining a copy
 #	of this software and associated documentation files (the "Software"), to deal
 #	in the Software without restriction, including without limitation the rights
 #	to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
 #	copies of the Software, and to permit persons to whom the Software is
 #	furnished to do so, subject to the following conditions:
 #
 #	The above copyright notice and this permission notice shall be included in all
 #	copies or substantial portions of the Software.
 #
 #	THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
 #	IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
 #	FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
 #	AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
 #	LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
 #	OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
 #	SOFTWARE.    
 
    
__DEBUG = False

treshold = 0.6 #lower the value for more noise, increase for less detection


#######################################################

parser = argparse.ArgumentParser(description='Understand functioning')
parser.add_argument("-v", "--verbose", help="increase output verbosity",action="store_true")
parser.add_argument('-x', action="store_true", default=False)
parser.add_argument('-y', action="store", dest="y")
parser.add_argument('-z', action="store", dest="z", type=int)
print (parser.parse_args(['-x', '-yval', '-z', '3']))
#if args.verbose:
#    print("verbosity turned on")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__) #<<<<<<<<<<<<<<<<<<<<

#https://github.com/starcraft04/swauto/blob/master/functions_opencv.py

def getMulti(res, tolerance,w,h):
    #We get an opencv image in the form of a numpy array and we need to
    #   find all the occurances in there knowing that 2 squares cannot intersect
    #This will give us exactly the matches that are unique
    
    #First we need to get all the points where value is >= tolerance
    #This wil get sometimes some squares that vary only from some pixels and that are overlapping
    all_matches_full = np.where (res >= tolerance)

    #Now we need to arrange it in x,y coordinates
    all_matches_coords = []
    for pt in zip(*all_matches_full[::-1]):
        all_matches_coords.append([pt[0],pt[1],res[pt[1]][pt[0]]])
    #Let's sort the new array
    all_matches_coords = sorted(all_matches_coords)
    
    #This function will be called only when there is at least one match so if matchtemplate returns something
    #This means we have found at least one record so we can prepare the analysis and loop through each records 
    all_matches = [[all_matches_coords[0][0],all_matches_coords[0][1],all_matches_coords[0][2]]]
    i=1
    for pt in all_matches_coords:
        found_in_existing = False
        for match in all_matches:
            #This is the test to make sure that the square we analyse doesn't overlap with one of the squares already found
            if pt[0] >= (match[0]-w) and pt[0] <= (match[0]+w) and pt[1] >= (match[1]-h) and pt[1] <= (match[1]+h):
                found_in_existing = True
                if pt[2] > match[2]:
                    match[0] = pt[0]
                    match[1] = pt[1]
                    match[2] = res[pt[1]][pt[0]]
        if not found_in_existing:
            all_matches.append([pt[0],pt[1],res[pt[1]][pt[0]]])
        i += 1

    #Before returning the result, we will arrange it with data easily accessible
    all_matches = getMultiFullInfo(all_matches,w,h) 
    return all_matches

    
def twoSquaresDoOverlap(squareA,squareB):
    #The two squares must have coordinates in the form of named list with name top_left and bottom_right
    overlap = True
    if squareA['top_left'][1] > squareB['bottom_right'][1] or \
            squareA['top_left'][0] > squareB['bottom_right'][0] or \
            squareA['bottom_right'][0] < squareB['top_left'][0] or \
            squareA['bottom_right'][1] < squareB['top_left'][1]:
        overlap = False
            
    return overlap


def cropToCoords(img, coords):
    (ulx,uly) = coords[0]
    (brx,bry) = coords[1]
    return img[uly:bry, ulx:brx]

def getMultiFullInfo(all_matches,w,h):
    #This function will rearrange the data and calculate the tuple
    #   for the square and the center and the tolerance for each point
    result = []
    for match in all_matches:
        tlx = match[0]
        tly = match[1]
        top_left = (tlx,tly)
        brx = match[0] + w
        bry = match[1] + h 
        bottom_right = (brx,bry)     
        centerx = match[0] + w/2
        centery = match[1] + h/2
        center = [centerx,centery]
        result.append({'top_left':top_left,'bottom_right':bottom_right,'center':center,'tolerance':match[2]})
    return result


def checkPicture(screenshot, templateFile, tolerance_list ,directories,allConfigs,multiple = False, showFound = False, saveFound = False):
    
    #This is an intermediary function so that the actual function doesn't include too much specific arguments

    if templateFile[:-4] in tolerance_list:
        tolerance = float(tolerance_list[templateFile[:-4]])
    else:
        tolerance = float(tolerance_list['global'])

    font = cv2.FONT_HERSHEY_PLAIN
    
    #Here we will detect all files that have the same base name ex: victory.png, victory02.png, ...
    allTemplateFiles = findAllPictureFiles(templateFile[:-4],directories['basepicsdir'])
    result = {}
    result['res'] = False
    result['best_val'] = 0
    result['points'] = []
    result['nameVersions'] = []
    result['name'] = templateFile
    
    for templateFileName in allTemplateFiles:
    
        template = cv2.imread(os.path.join(directories['basepicsdir'],templateFileName),-1)

        #The value -1 means we keep the file as is meaning with color and alpha channel if any
        #   btw, 0 means grayscale and 1 is color

        #Now we search in the picture
        result_temp = findPicture(screenshot,template, tolerance,allConfigs, multiple)
        
        if result_temp['res'] == True:
            if result['res'] == False:
                result['points'] = []
                result['nameVersions'] = []
            result['res'] = result_temp['res']
            result['best_val'] = result_temp['best_val']
            #!!!!! Attention, if the images are close to each other, there could be overlaps !!!!!!
            
            for result_temp_point in result_temp['points']:
                overlap = False
                for result_point in result['points']:
                    overlap = twoSquaresDoOverlap(result_point,result_temp_point)
                    if overlap:
                        break
                if not overlap:
                    result['points'].append(result_temp_point)
            result['nameVersions'].append(templateFileName)

        else:
            if result['res'] == False:
                result['best_val'] = result_temp['best_val']
                result['points'].extend(result_temp['points'])
                result['nameVersions'].append(templateFileName)
        
    #If it didn't get any result, we log the best value
    if not result['res']:
        print('Best value found for %s is: %f',templateFile,result['best_val'])
        color_showFound = (0,0,255)
    else:
        logging.info('Image %s found',templateFile)
        if logging.getLogger().getEffectiveLevel() == 10:
            saveFound = True
        color_showFound = (0,255,0)
            
    if saveFound or showFound:
        screenshot_with_rectangle = screenshot.copy()
        for pt in result['points']:
            cv2.rectangle(screenshot_with_rectangle, pt['top_left'], pt['bottom_right'], color_showFound, 2)
            fileName_top_left = (pt['top_left'][0],pt['top_left'][1]-10)
            cv2.putText(screenshot_with_rectangle,str(pt['tolerance'])[:4],fileName_top_left, font, 1,color_showFound,2)
        if saveFound:
            #Now we save to the file if needed
            filename = time.strftime("%Y%m%d-%H%M%S") + '_' + templateFile[:-4] + '.jpg'
            cv2.imwrite(os.path.join(directories['debugdir'] , filename), screenshot_with_rectangle)
        if showFound:
            cv2.imshow('showFound',screenshot_with_rectangle)
            cv2.waitKey(0)
    
    return result


def extractAlpha(img, hardedge = True):
    if img.shape[2]>3:
        print('Mask detected')
        channels = cv2.split(img)
        mask = np.array(channels[3])
        if hardedge:
            for idx in xrange(len(mask[0])):
                if mask[0][idx] <=128:
                    mask[0][idx] = 0
                else:
                    mask[0][idx] = 255

        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        return {'res':True,'image':img,'mask':mask}
    else:
        return {'res':False,'image':img}

    
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)    

def findPicture(screenshot,template, tolerance, multiple = False ):
    #This function will work with color images 3 channels minimum
    #The template can have an alpha channel and we will extract it to have the mask

    print('Tolerance to check is %f' , tolerance)
    print('*************Start of findPicture')

    h = template.shape[0]
    w = template.shape[1]
    
    #We will now extract the alpha channel
    tmpl = extractAlpha(template)    
    #print('alpha')
        
    # the method used for comparison, can be ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
    meth = 'cv2.TM_CCOEFF_NORMED'
    method = eval(meth)
    
    #print('method')

    # Apply template Matching
    if tmpl['res']:
        res = cv2.matchTemplate(screenshot,tmpl['image'],method, mask = tmpl['mask'])
    else:
        res = cv2.matchTemplate(screenshot,tmpl['image'],method)
        
        
    #print('res')    
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
        best_val = 1 - min_val
    else:
        top_left = max_loc
        best_val = max_val
    #We need to ensure we found at least one match otherwise we return false    
    #print('*best_val; ' + str(best_val) + " , tolerance:" + str(tolerance))   
    
    if best_val >= tolerance:
        if multiple:
            #We need to find all the time the image is found
            all_matches = getMulti(res, float(tolerance),int(w),int(h))
        else:
            bottom_right = (top_left[0] + w, top_left[1] + h)
            center = (top_left[0] + (w/2), top_left[1] + (h/2))
            all_matches = [{'top_left':top_left,'bottom_right':bottom_right,'center':center,'tolerance':best_val}]                        
         
        if __DEBUG == True:
            print('The points found will be:')
            print(all_matches)
            print('*************End of checkPicture')
            #return {'res': True,'best_val':best_val,'points':all_matches}                    
        return all_matches
    else:
        bottom_right = (top_left[0] + w, top_left[1] + h)
        center = (top_left[0] + (w/2), top_left[1] + (h/2))
        all_matches = [{'top_left':top_left,'bottom_right':bottom_right,'center':center,'tolerance':best_val}]

        print('Could not find a value above tolerance')
        print('*************End of findPicture')
        return {'res': False,'best_val':best_val,'points':all_matches}
    
# All the 6 methods for comparison in a list
#methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
#            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

# 	Point  	pt1,
#		Point  	pt2,
#		const Scalar &  	color,
#		int  	thickness = 1,
#		int  	lineType = LINE_8,
#		int  	shift = 0 
#'top_left': (139, 351), 'bottom_right': (176, 382), 'center': [157.5, 366.5], 'tolerance': 0.8956932}          
#images = convert_from_path('DEL_DU_3_SI_ESEC_----_04_GR_001_05-esec.pdf')

def runCounter(stackFile, needleFile, tresh, output_path=""):                
    stack_file_name = os.path.basename(stackFile ) 
    
    #read stackfile
    img_rgb = cv2.imread(stackFile)
    
    #grayImage
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    
    #needleFile
    template = cv2.imread(needleFile,1)
    
    #find matches
    matches = findPicture(img_rgb, template, tresh, multiple = True)   
    
    saveTextFile(matches, stack_file_name)   
    count=0
    rows = []
    json_data = {'objects': [{'a':'1'}]}
    
    #main loop, itterates through each coordinate found   
    for resultp in matches:
        count +=1 
        
        #draw rectangle
        cv2.rectangle(img_rgb, resultp['top_left'], resultp['bottom_right'], (0,0,255), 2)   
        
        #add counter text
        cv2.putText(img_rgb, str(count), (resultp['top_left'][0] ,resultp['top_left'][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 1)  
        
        if __DEBUG == True:
            print(str(count) + ": " + str(resultp['top_left']))  
            
        row = [count, resultp['top_left'][0], resultp['top_left'][1], resultp['bottom_right'][0], resultp['bottom_right'][1], resultp['center'][0], resultp['center'][1], resultp['tolerance'] ]        
        json_data['objects'].append( { "objectID": count , "top_left": [int(resultp['top_left'][0]),int(resultp['top_left'][1])], "bottom_right":[int(resultp['bottom_right'][0]),int(resultp['bottom_right'][1])], "center":[int(resultp['center'][0]),int(resultp['center'][1])], "tolerance": json.dumps(float(resultp['tolerance'])) } ) 
        
        rows.append(row)
        
    
    #save to csv
    saveCSV(rows, stack_file_name)
    saveJson(json_data, stack_file_name)
    
    #get x,y coordinates from stack image to print the counter text
    hi, wi, ci = img_rgb.shape   
    cv2.putText(img_rgb, "Count:" + str(count), (100, hi - 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 3)    

    #save processed images 
    output_path = os.path.join(output_path,stack_file_name + '_counter_.png')
    print("Save image to:" + output_path)    
    cv2.imwrite( output_path ,img_rgb)  
    #cv2.imshow(stackFile + '_counter_.png', img_rgb) #chrashed jupytier if count>1
    print("Finished") 

    
def saveTextFile(txt_data, txt_filename):   
    txt_output_path = os.path.join(csv_path,txt_filename + '.txt')
    print("Save Txt to:" + txt_output_path)    
    filewrite = open(txt_output_path,"w+")
    filewrite.write(str(txt_data))
    filewrite.close()
   
def saveCSV(data, csv_filename):
    csv_output_path = os.path.join(csv_path,csv_filename + '.csv')
    print("Save CSV to:" + csv_output_path)    
    fieldnames = ['objectID', 'top_left_x', 'top_left_y',  'bottom_right_x', 'bottom_right_y',  'center_x', 'center_y', 'tolerance' ]
    with open(csv_output_path, mode='w') as csv_file:        
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(fieldnames)              
        writer.writerows(data)
        
def saveJson(json_data, json_file_filename):
    #print("The type of object is: ", type(json_data))
    json_output_path = os.path.join(csv_path,json_file_filename + '.json')
    print("Save json to:" + json_output_path)        
    with open(json_output_path, 'w') as outfile:
      json.dump(json_data, outfile, ensure_ascii=False)    
        

        
###############################################              
              
cwd = os.getcwd()
stacks_path = os.path.join(cwd, 'data/stacks')
needle_path = os.path.join(cwd, 'data/needle')
output_path = os.path.join(cwd, 'data/output')
csv_path = os.path.join(cwd, 'data/output_csv')

        
if __DEBUG == True:
    print("stacks:" + stacks_path)    
    print("needle" + needle_path)    
    print("output:" + output_path)    
    print("csv_path:" + csv_path)    


for filename in os.listdir(stacks_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):        
        print("Process Stack file: " + os.path.join(stacks_path, filename))
        #run counter on each image with given needle
        #[path,needle,treshhold,debug]
        runCounter(os.path.join(stacks_path,filename),os.path.join(needle_path,'needle_4k.png'),treshold,output_path)
    else:
        print("No Files found in:" + output_path)
        continue

        

#https://appdividend.com/2019/07/18/python-list-example-list-in-python-tutorial-explained/
